In [6]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
from tensorflow.python.platform import gfile

In [ ]:
num_classes = 2
image_shape = (160, 576)
data_dir = './data'
runs_dir = './runs'
model_name = "./model/train.pb"
tests.test_for_kitti_dataset(data_dir)

input_tensor_name = 'image_input:0'
keep_prob_tensor_name = 'keep_prob:0'
logits_tensor_name = "logits:0"

with tf.Session() as sess:
    with gfile.FastGFile(model_name,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')
    
    input_image = sess.graph.get_tensor_by_name(input_tensor_name)
    keep_prob = sess.graph.get_tensor_by_name(keep_prob_tensor_name)
    logits = sess.graph.get_tensor_by_name(logits_tensor_name)
    
    helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)


Tests Passed
Training Finished. Saving test images to: ./runs\1544261242.2710128
